### Les indicateurs alimentaires permettent-il de mieux manger ?

## Introduction 

À l'heure où la société contemporaine connaît une période d'abondance alimentaire, les préoccupations environnementales croissantes et les impératifs de santé publique rendent parfois le choix de l'alimentation difficile pour le consomateur. Des indicateurs alimentaires sont alors apparus pour guider le public vers des produits jugées bénéfiques pour leur santé et l'état de la planète. Trois indicateurs se sont imposés en France, chacun considérant une dimension de la qualité d'un aliment. Le plus utilisé est le Nutriscore, qui est un indicateur nutritionel. L'Ecoscore renseigne sur l'impact environnemental et le score NOVA est informe sur le degré de transformation des aliments. 

Apparu au cours de la dernière décennie, ces indicateurs nutritionnels se sont révélés être des outils précieux des politiques publiques, qui ne rechignent pas à les utiliser. Ainsi l'affichage du nutriscore a été mis en place par le gouvernement français en 2016. Néanmoins la santé alimentaire ne semble pas s'être amélioré significativement depuis la mise en place de ces indicateurs. Cela nous amène à nous demander si ces indicateurs ont réellement un effet bénéfique sur le consommateur. 

Nous allons utiliser les données de la base Open Food Fact afin de mener une étude à ce sujet.
Dans un premier temps, nous allons télécharger la base et la nettoyer, afin de résoudre les problèmes liés à l'Open Data (Partie I). Puis nous avec l'aide d'analyses graphiques (Partie II), nous commencerons à formuler des hypothèses, auquelles nous répondrons à l'aide de modélisation (Partie III)

 





## Partie I : Récupération et nettoyage des données de la base Open Food Fact

# Partie I.1 : Présentation des données

# Partie I.1.1 Présentation Open Food Fact

présenter les données plus dico des variables 

Le Nutri-score est un système d'étiquetage nutritionnel à cinq niveaux, allant de A à E et du vert au rouge, placé sur le devant des emballages alimentaires, établi en fonction de la valeur nutritionnelle d'un produit alimentaire. Il a pour but d'aider les consommateurs à reconnaitre la qualité nutritionnelle globale des aliments et les aider à comparer les aliments entre eux, afin de favoriser le choix de produits plus favorable à la santé et ainsi de participer à la lutte contre les maladies chroniques comme les maladies cardiovasculaires, certains cancers, l'obésité et le diabète.

Nous souhaitons ici tenter de retrouver les principaux critères du Nutri-Score en regressant la valeur du nutri-score sur plusieurs variables qualitatives nutritionnelles. 
Puis nous aimerions élargir notre angle d'études en considérant d'autres critères pour quantifier la qualité d'un produit alimentaire (son niveau de transformation et sa provenance notamment). Nous crérons des scoring pour chacune des variables qu'on travaillerait. 
Enfin, nous aimerions mettre en évidence les différents catégories de produits alimentaires en utilisant des algorithmes de clustering à partir de nos scorings. 

Nous travaillerons sur la base de donnée OpenFoodFacts. Open Food Facts est un projet collaboratif dont le but est de constituer une base de données libre et ouverte sur les produits alimentaires commercialisés dans le monde entier. 
La première étape de notre projet est donc de nettoyer cette base de donnée très dense afin de pouvoir commencer nos analyses. 
Nous bornerons notre étude aux produits vendus en France, en ne gardant que les variables qui nous intéressent, pour cela on gardera les produits alimentaires qui auront toutes les variables d'intérêts renseignées.  

# Partie I.1.3 : Nova-Score et l'Ecosore

L'Éco-Score évalue l'impact environnemental des produits alimentaires en prenant en compte des critères écologiques variés tels que l'utilisation des terres, les émissions de gaz à effet de serre et la consommation d'eau. Il vise à sensibiliser les consommateurs aux implications environnementales de leurs choix alimentaires en attribuant une note qui permet de comparer la performance écologique des produits.



Le Score NOVA classe les produits alimentaires en fonction du degré de transformation qu'ils ont subi, allant de 1 à 4. Il prend en considération des critères tels que la nature des ingrédients, le degré de transformation, la présence d'additifs et la similitude avec des aliments non transformés. Cette classification encourage les consommateurs à opter pour des aliments moins transformés et plus proches de leur état naturel, mettant en avant les potentiels impacts sur la santé associés à la consommation d'aliments fortement transformés.


On importe les librairies Python qu'on utilisera dans le projet 

In [ ]:
#On importe les modules nécessaires au traitement de la base et communiquer les données

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#On importe les modules utilsés pour la modélisation


On importe la base de donnée OpenFoodFacts

In [ ]:
# Charger le fichier CSV
url_path = 'https://www.data.gouv.fr/fr/datasets/r/164c9e57-32a7-4f5b-8891-26af10f91072'
# Charger le fichier CSV dans un DataFrame pandas
df_openfoodfacts = pd.read_csv(url_path, sep='\t',low_memory=True)  # Assurez-vous de spécifier le bon séparateur s'il est différent de la virgule


On veut connaître ses dimensions avant nettoyage

In [ ]:
#On vérifie que la base est bien chargée
#on affiche 5 lignes aléatoires 
print(df_openfoodfacts.sample(5))

#on veut connaître la taille de la base
print ("Le dataframe compte {} lignes et {} variables".format(df_openfoodfacts.shape[0], df_openfoodfacts.shape[1]))


On commence le nettoyage de la base de donnée OpenFoodFacts: on ne garde que les produits qui sont vendus uniquement en France 

In [ ]:
# Obtention des valeurs distinctes de la colonne 'countries_tags' sans les valeurs nulles
valeurs_distinctes =df_openfoodfacts['countries_tags'].dropna().drop_duplicates().tolist()

# Tri des valeurs distinctes en ordre alphabétique
valeurs_distinctes.sort()

for valeur in valeurs_distinctes:
    print(valeur)

#On remarque que la France apparrait sous différentes formes, comme "fr:France", "en:France" ou encore "de:Frankreich".
#un peu de travail est nécéssaire pour récupérer tous les produits vendus en France




Nous avons obtenu la liste des pays où chaque produit est vendu. Au reagrd de la très grande diversité des produits et des pays de vente, nous avons décidé de se concentrer uniquement sur les produits vendus sur un seul marché, ici la France. 

Ce choix est pertinent pour plusieurs raisons . La France est en effet un pays où les étiquetages alimenataires sont très répandus et sont intégrés aux politiques publiques. De plus, les observations de produits français sont très présent dans la base Open food fact. Cela s'explique par l'origine hexagonale du projet et des campagnes de collectes de données menées par l'Agence Santé France.

En se focalisant sur les produits vendus en France, notre analyse se concentre de manière exclusive sur un marché unique. Cette approche garantit une comparaison pertinente entre les produits, facilitant ainsi l'interprétation des résultats. En outre, elle permet de maintenir la cohérence des données tout en assurant un volume suffisant pour des analyses approfondies.

On ne conserve ainsi que les produits uniquement vendus en France. 

In [ ]:
# Filtrer le DataFrame pour ne conserver que les lignes avec 'en:france' dans la colonne 'countries_tags'
df_france = df_openfoodfacts[df_openfoodfacts['countries_tags'] == 'en:france']
# Afficher les premières lignes du DataFrame résultant
print(df_france.head())

#on veut connaître la taille de la base
print ("Le dataframe compte {} lignes et {} variables".format(df_france.shape[0], df_france.shape[1]))

In [ ]:
# Filtrer le DataFrame pour ne conserver que les lignes avec 'en:france' dans la colonne 'countries_tags'
df_france = df_openfoodfacts[df_openfoodfacts['countries_tags'] == 'en:france']
# Afficher les premières lignes du DataFrame résultant
print(df_france.head())

#on veut connaître la taille de la base
print ("Le dataframe compte {} lignes et {} variables".format(df_france.shape[0], df_france.shape[1]))


On souhaite visualiser d'un coup d'oeil les variables d'intérêt de la base de donnée: quelles variables pourrons nous être utiles pour notre analyse? lesquelles sont assez remplies pour nous être utiles? 

In [ ]:
#On calcule le taux de remplissage de chaque variable
def null_factor(df):
  null_rate = ((df.isnull().sum() / df.shape[0])*100).sort_values(ascending=False).reset_index()
  null_rate.columns = ['Variable','Taux_de_Null']
  return null_rate

#Nous alllons désormais commencer à nettoyer la base de données en enlevant les colonnes peu remplis. 
filling_features = null_factor(df_france)
filling_features["Taux_de_Null"] = 100-filling_features["Taux_de_Null"]
filling_features = filling_features.sort_values("Taux_de_Null", ascending=False) 

#Seuil de suppression
sup_threshold = 10

#On affiche le taux de remplissages des variables en fonction d'un seuil de référence
fig = plt.figure(figsize=(20, 35))

font_title = {'family': 'serif',
              'color':  '#114b98',
              'weight': 'bold',
              'size': 18,
             }

sns.barplot(x="Taux_de_Null", y="Variable", data=filling_features, palette="flare")
#Seuil pour suppression des varaibles
plt.axvline(x=sup_threshold, linewidth=2, color = 'r')
plt.text(sup_threshold+2, 65, 'Seuil de suppression des variables', fontsize = 16, color = 'r')

plt.title("Taux de remplissage des variables dans le jeu de données (%)", fontdict=font_title)
plt.xlabel("Taux de remplissage (%)")
plt.show()

On va supprimer les colonnes qui ne sont pas remplies à + de 10% 

In [ ]:
#On ne décide de ne garder que les colonnes remplis à plus de 10%
seuil = 10  
filled_variables = list(filling_features.loc[filling_features['Taux_de_Null'] >= seuil, 'Variable'].values)

#Nouveau Dataset avec les variables conservées
df_france = df_france[filled_variables]

# Affichage du résultat
print ("Le dataframe df_france compte {} lignes et {} variables".format(df_france.shape[0], df_france.shape[1]))

for column_name in df_france.columns:
    print(column_name)

On allège de nouveau la base de donnée en enlevant les variables qui ne nous intéressent pas pour la suite (celles qui contiennent des images des produits notamment par exemple et qui consomment beaucoup de place)

In [ ]:
#On supprime les variables inutiles pour le reste du projet pour alléger la base
useless_columns = [col for col in df_france.columns if 'url' in col or 'image' in col or "categories" in col or "last" in col or "states" in col or "creator" in col ]
df_france = df_france.drop(columns=useless_columns)

print ("Le dataframe df_france compte {} lignes et {} variables".format(df_france.shape[0], df_france.shape[1]))

#On affiche le nom des colonnes restantes
for column_name in df_france.columns:
    print(column_name)


In [ ]:
#On ne garde que les observations où les informations nutritionneles sont complétées  


df_france = df_france[(df_france['nutriscore_grade'].notnull()) & (df_france['ecoscore_grade'].notnull()) & (df_france['fiber_100g'].notnull()) & (df_france['energy_100g'].notnull()) & (df_france['saturated-fat_100g'].notnull())  & (df_france['sugars_100g'].notnull())  & (df_france['proteins_100g'].notnull()) & (df_france['fat_100g'].notnull()) & (df_france['salt_100g'].notnull()) & (df_france['carbohydrates_100g'].notnull()) & (df_france['sodium_100g'].notnull())]

print(df_france.describe())

print ("Le dataframe df_france compte {} lignes et {} variables".format(df_france.shape[0], df_france.shape[1]))


In [ ]:
#On continue le nettoyage de la base en enlevant les valeurs abérrantes
#Open food fact est une base open source ouverte à tous. Il n'est pas abérrant de penser que des erreurs ont pu se produire lors  de l'entrée de certaines données

def suppression_aberrations(df):
    # Cette fonction supprime les observations où les valeurs sont aberrantes

    var_pour_100g = ['fat_100g', 'saturated-fat_100g', 'carbohydrates_100g', 'sugars_100g', 'fiber_100g', 'proteins_100g', 'salt_100g', 'sodium_100g']

    for var in var_pour_100g:
        df = df[(df[var] >= 0) & (df[var] <= 100)]

    return df


In [ ]:
#On regarde combien d'observations étaient abérrantes

ni=df_france.shape[0] 

# Appliquer la fonction sur le DataFrame
df_france = suppression_aberrations(df_france)

nf=df_france.shape[0]

delta_lignes=ni-nf #Controle du nombre de lignes supprimées 

print(delta_lignes,"lignes considérées comme des abérrations et donc supprimées")


In [ ]:
#Transformons le Nutriscore et l'ecoscore en variable quantitative, ce qui sera utile pour faire des régressions
def convertion_num_score(lettre):
    if lettre =="a":
        score=1
    elif lettre == "b":
        score=2
    elif lettre == "c":
        score=3
    elif lettre == "d":
        score=4
    elif lettre == "e":
        score=5
    else:
        score=np.nan
    return score

df_france["nutriscore_num"]=df_france['nutriscore_grade'].apply(convertion_num_score)
df_france["ecoscore_num"]=df_france['ecoscore_grade'].apply(convertion_num_score)

In [ ]:
#On affiche des informations sur la base finale

filling_features = null_factor(df_france)
filling_features["Taux_de_Null"] = 100-filling_features["Taux_de_Null"]
filling_features = filling_features.sort_values("Taux_de_Null", ascending=False) 
fig = plt.figure(figsize=(20, 35))

font_title = {'family': 'serif',
              'color':  '#114b98',
              'weight': 'bold',
              'size': 18,
             }

sns.barplot(x="Taux_de_Null", y="Variable", data=filling_features, palette="flare")
#Seuil pour suppression des varaibles
plt.axvline(x=sup_threshold, linewidth=2, color = 'r')
plt.text(sup_threshold+2, 65, 'Seuil de suppression des variables', fontsize = 16, color = 'r')

plt.title("Taux de remplissage des variables dans le jeu de données (%)", fontdict=font_title)
plt.xlabel("Taux de remplissage (%)")
plt.show()

In [ ]:
# Obtention des valeurs distinctes de la colonne 'countries_tags' sans les valeurs nulles
valeurs_distinctes = df_france['popularity_tags'].dropna().drop_duplicates().tolist()

# Tri des valeurs distinctes en ordre alphabétique
valeurs_distinctes.sort()

for valeur in valeurs_distinctes:
    print(valeur)

#On remarque que la France apparrait sous différentes formes, comme "fr:France", "en:France" ou encore "de:Frankreich".
#un peu de travail est nécéssaire pour récupérer tous les produits vendus en France




On réalise désormais des statistiques descriptives 
On commence par étudier le Nutriscore, qui est le principal score utilisé pour déterminer la qualitée nutritionnelles 

On commence par montrer la répartition du score du Nutriscore dans la base. 

In [ ]:

def plot_pie(score):
    # Filtrer les données pour exclure la catégorie 'Unknown' et 'non-applicable'

    filtered_data = df_france[(df_france[score] != 'unknown') & (df_france[score] != 'not-applicable')]

    # Compter les occurrences des valeurs dans la colonne 'nutriscore_grade' et trier par index (ordre alphabétique)
    score_counts = filtered_data[score].value_counts().sort_index()

    nutriscore_colors = {'a': '#2ecc71', 'b': '#55efc4', 'c': '#f4d03f', 'd': '#e67e22', 'e': '#e74c3c', '1.0': '#2ecc71', '2.0': '#f4d03f', '3.0': '#e67e22', '4.0': '#e74c3c'}
    colors = [nutriscore_colors.get(str(category), '#95a5a6') for category in score_counts.index]

    # Créer le diagramme circulaire avec les couleurs spécifiques du Nutri-Score
    plt.figure(figsize=(8, 8))
    plt.pie(score_counts, labels=score_counts.index, autopct='%1.1f%%', startangle=9, colors=colors)
    score_name = score.split("_")
    plt.title('Répartition du score ' + score_name[0])
    plt.show()


#On affiche les différentes scores
plot_pie("nutriscore_grade")
plot_pie("ecoscore_grade")
plot_pie("nova_group")

In [ ]:
def time_evolution(score):

    # Make sure the 'created_datetime' column is in datetime format
    df_france['created_datetime'] = pd.to_datetime(df_france['created_datetime'])

    # Filtrer les données pour exclure la catégorie 'Unknown' et 'non-applicable'
    filtered_data = df_france[(df_france[score] != 'unknown') & (df_france[score] != 'not-applicable')]

    # Make sure the 'created_datetime' column is in datetime format
    df_france['created_datetime'] = pd.to_datetime(df_france['created_datetime'])


    # Extract year from the 'created_datetime' column
    filtered_data['year'] = filtered_data['created_datetime'].dt.year

    # On crée un nouveau dataframe pour manipuler les données utiles pour le graphique
    cumulative_df = pd.DataFrame(index=filtered_data['year'].unique(), columns=filtered_data[score].unique())

    # On calcule le nombre d'occurences par note et par an 
    for year in cumulative_df.index:
        year_data = filtered_data[filtered_data['year'] <= year]
        counts = year_data[score].value_counts(normalize=True) * 100  # Calculate percentages
        cumulative_df.loc[year] = counts

    # On affiche les années dans l'ordre
    cumulative_df.sort_index(inplace=True)
    cumulative_df = cumulative_df[sorted(cumulative_df.columns)]


    #On affiche le graphique 
    nutriscore_colors = {'a': '#2ecc71', 'd': '#55efc4', 'c': '#f4d03f', 'b': '#e67e22', 'e': '#e74c3c',
                         '1.0': '#2ecc71', '2.0': '#f4d03f', '3.0': '#e67e22', '4.0': '#e74c3c'}

    colors = [nutriscore_colors.get(str(category), '#95a5a6') for category in counts.index]

    cumulative_df.plot(kind='bar', stacked=True, figsize=(10, 6), color=colors)
    
    #On affiche le titre et la légende
    score_name = score.split("_")
    plt.title('Répartition du score ' + score_name[0]) 
    plt.legend(title='Grade', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.ylabel('Pourcentage')
    plt.xlabel('Années')
    plt.yticks(range(0, 101, 10), [f'{i}%' for i in range(0, 101, 10)])

    plt.show()

time_evolution("nutriscore_grade")
time_evolution("ecoscore_grade")
time_evolution("nova_group")



In [ ]:
def new_creation_time(score):

    filtered_data = df_france[(df_france[score] != 'unknown') & (df_france[score] != 'not-applicable')]

    score_per_year = filtered_data[['code', score]].groupby(by=[score,filtered_data['created_datetime'].dt.year]).nunique().reset_index()
    cum_per_year = filtered_data[['code']].groupby(by=filtered_data['created_datetime'].dt.year).nunique().reset_index()
    score_per_year = pd.merge(score_per_year, cum_per_year, how="left", left_on="created_datetime", right_on="created_datetime")
    score_per_year = score_per_year.rename(columns={'created_datetime':'year', 'code_x':'nb_score', 'code_y':'total_grade'})
    score_per_year['score_rate'] = (score_per_year['nb_score'] / score_per_year['total_grade'])*100


    nutriscore_colors = {'a': '#2ecc71', 'd': '#55efc4', 'c': '#f4d03f', 'b': '#e67e22', 'e': '#e74c3c',
                         '1.0': '#2ecc71', '2.0': '#f4d03f', '3.0': '#e67e22', '4.0': '#e74c3c'}
    
    colors = [nutriscore_colors.get(str(category), '#95a5a6') for category in score_per_year[score].index]

    fig =plt.figure(figsize=(12,8))
    ax = sns.lineplot(x='year', y='score_rate', hue=score, data=score_per_year, palette = colors)
    plt.xlabel("Année")
    plt.ylabel("Taux")

    score_name = score.split("_")
    plt.title('2volution des entrées des nouveaux scores ' + score_name[0]) 
    
    plt.show()

new_creation_time("nutriscore_grade")

In [ ]:
#On affiche la répartition du score brut du nutriscore, avant transformation en grade

# Filtrer les données pour exclure la catégorie 'Unknown' et 'non-applicable'
filtered_data = df_france[(df_france["nutriscore_grade"] != 'unknown') & (df_france["nutriscore_grade"] != 'not-applicable')]

    
fig, axes = plt.subplots(1, 2, sharex=False, sharey=False, figsize=(21,8))
fig.suptitle("Répartition des scores Nutriscore et de leurs grades" "\n", fontdict=font_title)

sns.histplot(data=filtered_data.sort_values("nutriscore_grade"), x="nutriscore_grade", hue="nutriscore_grade", ax=axes[0])
axes[0].set_title('Grades de Nutriscores')
axes[0].set_xlabel("Grade Nutriscore")
axes[0].set_ylabel("Nombre de produits")


sns.histplot(data=filtered_data.sort_values("nutriscore_grade"), x="nutriscore_score", hue="nutriscore_grade", ax=axes[1])
axes[1].set_title('Scores de Nutriscores')
axes[1].set_xlabel("Score Nutriscore")
axes[1].set_ylabel("Nombre de produits")

plt.show()

On étudie la qualité de la nourriture sous 3 aspects : écologie, nutritionelles et transformations. 
On va utiliser une heatmap pour voir s'il n'existe pas déjà des corrélations entre les variables




In [ ]:
variables = ['additives_n','energy_100g', 'fat_100g'
        , 'saturated-fat_100g', 'carbohydrates_100g',
       'sugars_100g', 'fiber_100g', 'proteins_100g', 'salt_100g',
       'sodium_100g', 'nutriscore_num', 'ecoscore_num', 'nova_group']

x=df_france.loc[:, variables]
x = x.dropna()
 
#calculate the correlation matrix
corr = x.corr()

lab=x.columns
#plot the heatmap
fig=plt.figure(figsize=[12,9])
fig.patch.set_facecolor('#E0E0E0')
fig.patch.set_alpha(0.7)

plt.title("Correlation linéaire entre les variables",size=18)
ax=sns.heatmap(corr, vmin=-1, vmax=1,cmap="bwr",
        xticklabels=variables,
        yticklabels=variables)


In [ ]:
II. Partie analyse

On va maintenant créer une base de donnée unifiée qui ne contient que les variables d'intérêt pour la suite où toutes les occurences pour les scorings créés après sont renseignés 

In [ ]:
# Filtrer les lignes où 'labels_tags' et 'ingredients_text' ne sont pas nulles
df_filtered = df_cleaned.dropna(subset=['labels_tags', 'ingredients_text'])

# Sélectionner les colonnes spécifiques
selected_columns = ['code', 'countries_tags', 'ecoscore_grade', 'nutriscore_grade', 'product_name',
                     'energy_100g', 'saturated-fat_100g', 'sugars_100g', 'proteins_100g',
                     'fat_100g', 'carbohydrates_100g', 'energy-kcal_100g', 'sodium_100g', 'salt_100g',
                     'food_groups_tags', 'labels_tags', 'nutriscore_score', 'nutrition-score-fr_100g',
                     'ecoscore_score', 'ingredients_text', 'nova_group', 'fiber_100g']

# Créer le DataFrame final
df_selected = df_filtered[selected_columns]

# Afficher le DataFrame résultant
df_selected



In [ ]:
print ("Le dataframe df_cleaned compte {} lignes et {} variables".format(df_selected.shape[0], df_selected.shape[1]))


A partir de cette liste, en modifiant à la main, on va garder uniquement les variables que je souhaite étudier pour la suite. 

On va désormais élargir notre analyse en prenant en compte de nouveaux critères qui sont importants lorsqu'on souhaite mieux s'alimenter: le degré de transformation, les additifs et la provenance des produits. On va essayer de créé des variables de scoring pour ces trois catégories avant de faire une comparaison avec le Nutri-Score étudié au dessus. 

1. Analyse du degré de transformation

Pour cette variable, le principal problème est la quantification de la transformation du produit alimentaire. Une approche simple serait de considérer que plus un produit alimentaire contient d'ingrédient, plus il est transformé. 

En s'appuyant sur les critères du nova-score, on va essayer de reproduire son scoring en appliquant les critères décrits ici: https://scanup.fr/degre-de-transformation-des-aliments-la-classification-nova/ .

D'abord, on va donc identifier les produits de la base qui correspondent à des produits naturels (fruits, légumes, poisson qui sont tels quels) en utilisant la catégorie food_groups_tags, avant de faire une analyse des ingrédients sur les produits qu'on identifie comme étant non bruts. 
On observe sur le graphique généré précédemment qu'environ 30% des produits ont la variable ingredient_text renseigné, ce qui nous permettra de générer un nova score sur environ 30% des produits de la base. 
La variable food_groups_tags nous permettra de corriger les cas déviants les plus problématiques après la première étape de scoring. 

In [ ]:
# Afficher les valeurs uniques de la colonne 'main_category_fr'
valeurs_main_category_fr = df_selected['food_groups_tags'].unique()

# Afficher les valeurs
print(valeurs_main_category_fr)


On fait un premier test de scoring en prenant en compte uniquement le nombre d'ingédients listés dans le produit. On veut d'abord regarder comment la variable est renseignée pour choisir le meilleur séparateur d'ingrédiant qui pourra nous permettre de trouver le plus justement possible le nombre d'ingrédents sachant qu'à posteriori, on a vu que la variable n'était pas renseignée de manière homogène. 

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt

# Assume df_france is your DataFrame with numerical features
# Select relevant columns and handle missing values if necessary
selected_columns = ['additives_n', 'energy_100g', 'fat_100g',
                    'saturated-fat_100g', 'carbohydrates_100g',
                    'sugars_100g', 'fiber_100g', 'proteins_100g', 'salt_100g',
                    'sodium_100g', 'nutriscore_num', 'ecoscore_num', 'nova_group']
df_selected = df_france[selected_columns].dropna()

# Standardize the data
scaler = StandardScaler()
df_selected_scaled = scaler.fit_transform(df_selected)

# Use DBSCAN for clustering
dbscan = DBSCAN(eps=0.5, min_samples=5)  # Adjust eps and min_samples based on your data
clusters = dbscan.fit_predict(df_selected_scaled)

# Visualize the clusters (2D example)
plt.scatter(df_selected_scaled[:, 0], df_selected_scaled[:, 1], c=clusters, cmap='viridis')
plt.title('DBSCAN Clustering')
plt.xlabel('Feature 1 (Standardized)')
plt.ylabel('Feature 2 (Standardized)')
plt.show()


In [ ]:
# Afficher les valeurs uniques de la colonne 'main_category_fr'
valeurs_main_category_fr = df_selected['ingredients_text'].unique()

# Afficher les valeurs
print(valeurs_main_category_fr)


On voit que le renseignement de la variable ingredients_text est assez hétérogène: On a donc mis en évidence plusieurs manières de comptabiliser le nb d'ingrédients:
- en comptant le nombre de virgules dans la chaîne de caractère
- en comptant le nombre d'espaces dans la chaîne de caractère (plus fiable mais tend à augumenter le nb d'ingrédient par rapport à la réalité)
- le nombre de points virgule

Finalement, compter les espaces est trop hazardeux, donc on va se concentrer sur le comptage des virgules. On décide donc finalement de compter la somme des virgules et des points virgules puisqu'en général, les deux séparateurs ne sont pas utilisés simultanément, donc compter leur somme nous permettra de balayer le plus de liste d'ingrédients possible tout en faussant le moins possible. 

Il nous restera à traiter ensuite du cas des listes d'ingrédients sans séparateurs. 


In [ ]:
# Compter le nombre d'espaces dans chaque chaîne de caractères de 'ingredients_text'
df_selected['nombre_ingredients'] = df_selected['ingredients_text'].apply(lambda x: x.count(',') + x.count(';'))

# Afficher le DataFrame avec la nouvelle colonne
print(df_selected[['ingredients_text', 'nombre_ingredients']])

# Obtenir un tableau d'occurrences du nombre d'ingrédients
occurrences_nb_ingredients = df_selected['nombre_ingredients'].value_counts()

# Afficher le tableau d'occurrences
print(occurrences_nb_ingredients)

On va établir le scoring suivant: 
- produit brut: 1 si le nombre d'ingrédients est égal à 1 ou 2; 
- ingrédient culiaire: 2 si le nom d'ingrédients est 3 ou 4; 
- produit simplement transformé: 3 si le nombre d'ingrédiants est situé entre 5 et 7 et 
- produit très transformé: 4 si le nombre d'ingrédients est au delà de 8. 

In [ ]:
# Créer une nouvelle colonne 'score' en fonction du nombre d'ingrédients
df_selected['score_transformation'] = df_selected['nombre_ingredients'].apply(lambda n: 1 if n <= 2 else (2 if n <= 4 else (3 if n <= 7 else 4)))

# Afficher le DataFrame avec la nouvelle colonne 'score'
#print(df_selected[['ingredients_text', 'nombre_ingredients', 'score_transformation']])

# Obtenir un tableau d'occurrences du scoring sur la transformation des produits
occurrences_scoring_transformation = df_selected['score_transformation'].value_counts()

# Afficher le tableau d'occurrences
print(occurrences_scoring_transformation)


Pour tester le scoring qu'on vient de créer, nous proposons de regarder les occurences de food_groups_tags qu'on retrouve parmi les produits qu'on a classé en scoring 1 pour la transformation pour voir s'il y a une certaine cohérence (nous ne sommes pas censé retrouver des buscuits par exemple)

In [ ]:
# Filtrer les produits avec un score de 1
score_1_products = df_selected[df_selected['score_transformation'] == 1]

# Afficher les occurrences de food_groups_tags pour les produits avec un score de 1
occurrences_score_1 = score_1_products['food_groups_tags'].value_counts()

# Afficher les résultats
print(occurrences_score_1)


On va vérifier les occurences étranges et voir ce qu'on peut faire pour corriger ces défauts 

In [ ]:
# Filtrer les produits où food_groups_tags prend la valeur "en:sugary-snacks,en:biscuits-and-cakes"
filtered_products = score_1_products[score_1_products['food_groups_tags'] == 'en:sugary-snacks,en:biscuits-and-cakes']

# Afficher la colonne 'product_name' des produits filtrés
print(filtered_products[['product_name', 'ingredients_text']])


Finalement, pour régler le problème des listes d'ingrédients sans séparateurs, on va utiliser la catégorie des produits transformés renseigné dans food_groups_tags pour les mettre directement en score 4, ce qui permettra d'enlever ces passagers clandestins des catégories 1, 2 et 3. 

In [ ]:
# Liste des catégories de produits transformés
categorie_produits_transformes = ['en:composite-foods,en:one-dish-meals',
                                  'en:composite-foods,en:pizza-pies-and-quiches',
                                  'en:sugary-snacks,en:biscuits-and-cakes',
                                  'en:salty-snacks,en:salty-and-fatty-products',
                                  'en:salty-snacks,en:appetizers',
                                  'en:composite-foods,en:sandwiches',
                                  'en:sugary-snacks,en:chocolate-products']

# Mise à jour du score pour les produits dans la liste des catégories de produits transformés
df_selected.loc[df_selected['food_groups_tags'].isin(categorie_produits_transformes), 'score_transformation'] = 4

# Afficher le DataFrame mis à jour
print(df_selected[['product_name', 'food_groups_tags', 'score_transformation']])
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

# Assume df_france is your DataFrame with the relevant columns
# Select relevant columns and handle missing values if necessary
selected_columns = ['nova_group', 'nutriscore_num', "ecoscore_num"]
df_selected = df_france[selected_columns].dropna()

# Define the independent variable (X) and the dependent variable (y)
X = df_selected[['nutriscore_num', "ecoscore_num"]]
y = df_selected['nova_group']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a linear regression model
model = LinearRegression()

# Fit the model to the training data
model.fit(X_train, y_train)

print("Intercept:", model.intercept_)
print("Coefficient for 'nutriscore_num':", model.coef_)



On refait un test pour voir si les scorings 1 clandestins ont été corrigés

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt

# Assume df_france is your DataFrame with the relevant columns
# Select relevant columns and handle missing values if necessary
selected_columns = ['nova_group', 'nutriscore_num']
df_selected = df_france[selected_columns].dropna()

# Define the independent variable (X) and the dependent variable (y)
X = df_selected[['nutriscore_num']]
y = (df_selected['nova_group'] > 0).astype(int)  # Convert to binary outcome (0 or 1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a logistic regression model
logreg_model = LogisticRegression()

# Fit the model to the training data
logreg_model.fit(X_train, y_train)

# Plot the coefficients
plt.bar(X.columns, logreg_model.coef_[0])
plt.xlabel('Variable')
plt.ylabel('Coefficient')
plt.title('Coefficients of Logistic Regression')
plt.show()


In [ ]:
# Filtrer les produits avec un score de 1
score_1_products = df_selected[df_selected['score_transformation'] == 1]

# Afficher les occurrences de food_groups_tags pour les produits avec un score de 1
occurrences_score_1 = score_1_products['food_groups_tags'].value_counts()

# Afficher les résultats
print(occurrences_score_1)

2. Origine des aliments 

On va maintenant s'intéresser aux labels des produits alimentaires qu'on considère grâce à la variable labels_tags et tenter de faire une variable de scoring qui nous permettrait de quantifier sa qualité en fonction des labels qu'on lui a accordé quant à sa provenance ou sa production (étiquette bio ou made in France notamment)

In [ ]:
# Afficher les occurrences qui apparaissent plus de 1000 fois dans la colonne 'labels_tags'
occurrences_plus_de_1000 = df_selected['labels_tags'].value_counts()[df_selected['labels_tags'].value_counts() > 1000]

# Afficher le tableau d'occurrences
print(occurrences_plus_de_1000)


On va considérer le scoring suivant pour quantifier l'origine et la provenance des produits alimentaires. 
On se basera sur la provenance (France/UE) et la catégorie de bio 
On définera les scores suivants:
- 1 si le produit est français et bio 
- 2 si le produit est européen et bio 
- 3 si le produit est français
- 4 si le produit est bio 
- 5 sinon 

On a choisi de faire du plus petit au plus grand pour décrire l'évolution du "meilleur" au "pire" pour se caler sur la logique du nutriscore, ce qui nous facilitera les comparaisons par la suite. 

Pour mesurer les différents attributs, on va faire uen analyse dans les chaînes de caractère des labels: s'il y a france ou fr on lui attribuera le fait qu'il est français par exemple. 

In [ ]:
# Créer une nouvelle colonne 'score_labels' initialisée à 0
df_selected['score_labels'] = 0

# Définir des critères et attribuer des scores
critere_bio_france = df_selected['labels_tags'].str.contains('france|fr', case=False) & df_selected['labels_tags'].str.contains('bio|organic', case=False)
critere_eu_bio = df_selected['labels_tags'].str.contains('eu', case=False) & df_selected['labels_tags'].str.contains('bio|organic', case=False)
critere_france = df_selected['labels_tags'].str.contains('france|fr', case=False)
critere_bio = df_selected['labels_tags'].str.contains('bio|organic', case=False)

# Attribuer des scores en fonction des critères (inverser l'ordre)
df_selected.loc[~(critere_bio_france | critere_eu_bio | critere_france | critere_bio) & (df_selected['score_labels'] < 1), 'score_labels'] = 5
df_selected.loc[critere_bio & (df_selected['score_labels'] < 2), 'score_labels'] = 4
df_selected.loc[critere_france & (df_selected['score_labels'] < 3), 'score_labels'] = 3
df_selected.loc[critere_eu_bio & (df_selected['score_labels'] < 4), 'score_labels'] = 2
df_selected.loc[critere_bio_france, 'score_labels'] = 1

# Afficher le DataFrame avec la nouvelle colonne de scoring
print(df_selected[['labels_tags', 'score_labels']])


In [ ]:
df_selectedprint(df_selected_scaled_df.shape)